In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp -r /content/gdrive/My\ Drive/whiteboard_object_detection/weights/model_final_1500.pth /content

# !cp -r /content/gdrive/My\ Drive/whiteboard_object_detection/videos/n1.MTS /content
!cp -r /content/gdrive/My\ Drive/whiteboard_object_detection/videos/n1.MTS /content
# !cp -r /content/gdrive/My\ Drive/whiteboard_object_detection/videos/1.mp4 /content





In [1]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 24kB/s 
     |████████████████████████████████| 4.1MB 53.5MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
     |████████████████████████████████| 276kB 6.6MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=71ddf6d3b13f7f2e3fe47b095d8cf704392937282b59c261a103d37bde1e7d5d
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-vsddk4gk
  Running command git 

In [2]:
import torch, torchvision
torch.__version__
!gcc --version

gcc (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
     |████████████████████████████████| 6.2MB 875kB/s 
     |████████████████████████████████| 61kB 3.8MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.dev200316-cp36-none-any.whl size=38421 sha256=654eeb893805e446b7044a754d558b54d492815dc27ff01edc5312b67da400fa
  Stored in directory: /root/.cache/pip/wheels/d1/97/bd/20ecf786af30dffc3bf59471165e151ef823d97feadb3221aa
Successfully built fvcore
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import cv2
import time
import os


def video_to_frames(input_loc):

    time_start = time.time()
    cap = cv2.VideoCapture(input_loc)
    print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print("Number of frames: ", video_length)
    count = 0
    fps = round(cap.get(cv2.CAP_PROP_FPS))
    print(fps)
    print("Converting video..\n")
    all_frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if (count % fps == 0):
            all_frames.append(frame)

        count = count + 1
        if (count > (video_length - 1) ):
            time_end = time.time()
            cap.release()
            print("Done extracting frames.\n%d frames extracted" % count)
            print("It took %d seconds for conversion." % (time_end - time_start))

            break
    return all_frames


In [14]:
all_frames=video_to_frames("/content/n1.MTS")

23026.0
Number of frames:  23025
30
Converting video..

Done extracting frames.
23025 frames extracted
It took 158 seconds for conversion.


In [0]:
import cv2
import torch
import os
import time
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor


def get_predictor():

    cfg = get_cfg()
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    cfg.MODEL.WEIGHTS = "model_final_1500.pth"
    return DefaultPredictor(cfg)


def get_annotations(all_frames=[]):
    time_start=time.time()
    annotations=[]
    predictor=get_predictor()
    for second,frame in enumerate(all_frames):
        outputs = predictor(frame)
        boxes=[]
        for pred_box in outputs["instances"].pred_boxes:
            pred_box = pred_box.int()
            box = [pred_box[0].item(), pred_box[1].item(), pred_box[2].item(), pred_box[3].item()]
            boxes.append(box)
        annotations.append(boxes)
    time_end=time.time()
    print("It took %d seconds for annotation." % (time_end - time_start))

    return annotations



In [15]:
annotations= get_annotations(all_frames)

It took 277 seconds for annotation.


In [0]:
import pickle
pickle_out = open("n1.pickle", "wb")
pickle.dump(annotations, pickle_out)
pickle_out.close()

In [17]:
!nvidia-smi

Mon Mar 16 17:58:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    62W / 149W |   2921MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  